In [1]:
import math
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import sys
from glob import glob
from hydra import initialize, compose
from matplotlib import colors
from pprint import pprint

# set paths to various directories based on the machine this code is being executed on
with initialize(version_base=None, config_path='config'):
    config = compose(config_name='config.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, figure_dir, pickle_dir, repo_dir = config.machine.array_dir, config.machine.data_dir, config.machine.figure_dir, config.machine.pickle_dir, config.machine.repo_dir

# enable use of local modules
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

# set matplotlib style
plt.style.use(f'{repo_dir}/mejiro/mplstyle/science.mplstyle')

from mejiro.lenses.test import SampleStrongLens
from mejiro.plots import diagnostic_plot, plot, plot_util, overplot
from mejiro.analysis import stats
from mejiro.utils import util
from mejiro.helpers import pyhalo, pandeia_input

In [2]:
input_dir = config.machine.dir_03

In [3]:
pipeline_params = util.hydra_to_dict(config.pipeline)
bands = pipeline_params['bands']
pieces = pipeline_params['pieces']

In [4]:
lens_pickles = sorted(glob(config.machine.dir_02 + '/lens_with_subhalos_*.pkl'))
lens_uids = [int(os.path.basename(i).split('_')[3].split('.')[0]) for i in lens_pickles]
uid = lens_uids[0]
lens = util.unpickle(os.path.join(input_dir, f'lens_{str(uid).zfill(8)}.pkl'))

In [5]:
pprint(sorted(glob(f'{input_dir}/array_{lens.uid}_*.npy')))

['/data/scratch/btwedig/mejiro/pipeline/03/array_00000000_F087.npy',
 '/data/scratch/btwedig/mejiro/pipeline/03/array_00000000_F106.npy',
 '/data/scratch/btwedig/mejiro/pipeline/03/array_00000000_F129.npy',
 '/data/scratch/btwedig/mejiro/pipeline/03/array_00000000_F158.npy',
 '/data/scratch/btwedig/mejiro/pipeline/03/array_00000000_lens_F087.npy',
 '/data/scratch/btwedig/mejiro/pipeline/03/array_00000000_lens_F106.npy',
 '/data/scratch/btwedig/mejiro/pipeline/03/array_00000000_lens_F129.npy',
 '/data/scratch/btwedig/mejiro/pipeline/03/array_00000000_lens_F158.npy',
 '/data/scratch/btwedig/mejiro/pipeline/03/array_00000000_source_F087.npy',
 '/data/scratch/btwedig/mejiro/pipeline/03/array_00000000_source_F106.npy',
 '/data/scratch/btwedig/mejiro/pipeline/03/array_00000000_source_F129.npy',
 '/data/scratch/btwedig/mejiro/pipeline/03/array_00000000_source_F158.npy']


In [6]:
arrays = [np.load(f'{input_dir}/array_{lens.uid}_{band}.npy') for band in bands]
if pieces:
    lens_sbs = [np.load(f'{input_dir}/array_{lens.uid}_lens_{band}.npy') for band in bands]
    source_sbs = [np.load(f'{input_dir}/array_{lens.uid}_source_{band}.npy') for band in bands]
    arrays += lens_sbs
    arrays += source_sbs
    bands = bands * 3

In [8]:
# print(len(lens_sbs), len(source_sbs))

In [9]:
j = 0
for i, (band, arrays) in enumerate(zip(bands, arrays)):
    if j == 0:
        print('array')
    elif j == 1:
        print('lens')
    elif j == 2:
        print('source')
    if i % len(set(bands)) == len(set(bands)) - 1:
        j += 1

array
array
array
array
